In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

In [2]:
data_dir = '../data'
ckpt_dir = '../checkpoint'
word_embd_dir = '../checkpoint/word_embd'
model_dir = '../checkpoint/modelv3'

In [3]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
word_state_size = 100
other_state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 10

In [4]:
with tf.name_scope("input"):
    path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length")
    word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size], name="y")

In [5]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [6]:
with tf.name_scope("dropout"):
    embedded_word_drop = tf.nn.dropout(embedded_word, 0.3)

In [7]:
word_hidden_state = tf.zeros([batch_size, word_state_size], name='word_hidden_state')
word_cell_state = tf.zeros([batch_size, word_state_size], name='word_cell_state')
word_init_state = tf.contrib.rnn.LSTMStateTuple(word_hidden_state, word_cell_state)

other_hidden_states = tf.zeros([channels-1, batch_size, other_state_size], name="hidden_state")
other_cell_states = tf.zeros([channels-1, batch_size, other_state_size], name="cell_state")

other_init_states = [tf.contrib.rnn.LSTMStateTuple(other_hidden_states[i], other_cell_states[i]) for i in range(channels-1)]

with tf.variable_scope("word_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word_drop[0], sequence_length=path_length[0], initial_state=word_init_state)
    state_series_word1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("word_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word_drop[1], sequence_length=path_length[1], initial_state=word_init_state)
    state_series_word2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=other_init_states[0])
    state_series_pos1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=other_init_states[0])
    state_series_pos2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=other_init_states[1])
    state_series_dep1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=other_init_states[1])
    state_series_dep2 = tf.reduce_max(state_series, axis=1)

state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

state_series = tf.concat([state_series1, state_series2], 1)

In [9]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer, W) + b
    predictions = tf.argmax(logits, 1)

In [ ]:
tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('biases')==-1):
            tv_regu.append(t)

with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

global_step = tf.Variable(0, name="global_step")

optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

In [10]:
f = open(data_dir + '/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

pos_tags_vocab = []
for line in open(data_dir + '/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open(data_dir + '/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open(data_dir + '/relation_types.txt'):
    relation_vocab.append(line.strip())


rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [12]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()

# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [18]:
# model = tf.train.latest_checkpoint(model_dir)
# saver.restore(sess, model)

In [19]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd/word_embd


In [15]:
f = open(data_dir + '/train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open(data_dir + '/train_relations.txt'):
    relations.append(line.strip().split()[1])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [20]:
num_epochs = 20
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        if step%10==0:
            print("Step:", step, "loss:",loss)
        if step % 1000 == 0:
            saver.save(sess, model_dir + '/model')
            print("Saved Model")

Step: 10 loss: 2.58496
Step: 20 loss: 3.10737
Step: 30 loss: 2.76529
Step: 40 loss: 2.86065
Step: 50 loss: 2.87625
Step: 60 loss: 2.70682
Step: 70 loss: 2.63103
Step: 80 loss: 2.64477
Step: 90 loss: 2.60822
Step: 100 loss: 2.66668
Step: 110 loss: 2.41839
Step: 120 loss: 2.55943
Step: 130 loss: 2.3208
Step: 140 loss: 2.14755
Step: 150 loss: 2.53709
Step: 160 loss: 2.61406
Step: 170 loss: 2.29024
Step: 180 loss: 2.43227
Step: 190 loss: 2.44584
Step: 200 loss: 2.57538
Step: 210 loss: 2.40568
Step: 220 loss: 2.267
Step: 230 loss: 2.50777
Step: 240 loss: 1.94963
Step: 250 loss: 1.83509
Step: 260 loss: 2.26527
Step: 270 loss: 2.09075
Step: 280 loss: 1.53659
Step: 290 loss: 2.21628
Step: 300 loss: 1.84958
Step: 310 loss: 1.6308
Step: 320 loss: 1.86496
Step: 330 loss: 1.96894
Step: 340 loss: 2.60143
Step: 350 loss: 3.13792
Step: 360 loss: 2.17429
Step: 370 loss: 1.34178
Step: 380 loss: 1.69578
Step: 390 loss: 2.02371
Step: 400 loss: 1.84938
Step: 410 loss: 2.09912
Step: 420 loss: 1.61329
Step:

Step: 3310 loss: 1.29887
Step: 3320 loss: 1.29867
Step: 3330 loss: 0.443141
Step: 3340 loss: 0.454681
Step: 3350 loss: 1.124
Step: 3360 loss: 1.11588
Step: 3370 loss: 0.939808
Step: 3380 loss: 1.22224
Step: 3390 loss: 1.02613
Step: 3400 loss: 1.35341
Step: 3410 loss: 0.489673
Step: 3420 loss: 0.418773
Step: 3430 loss: 1.12018
Step: 3440 loss: 0.651813
Step: 3450 loss: 0.940705
Step: 3460 loss: 1.46314
Step: 3470 loss: 0.754336
Step: 3480 loss: 1.34626
Step: 3490 loss: 0.632453
Step: 3500 loss: 0.942931
Step: 3510 loss: 1.01058
Step: 3520 loss: 0.921788
Step: 3530 loss: 0.574035
Step: 3540 loss: 1.17512
Step: 3550 loss: 2.10246
Step: 3560 loss: 1.22662
Step: 3570 loss: 0.783618
Step: 3580 loss: 1.29106
Step: 3590 loss: 1.17994
Step: 3600 loss: 1.00297
Step: 3610 loss: 1.47327
Step: 3620 loss: 0.591666
Step: 3630 loss: 0.624228
Step: 3640 loss: 0.619013
Step: 3650 loss: 0.970902
Step: 3660 loss: 1.09406
Step: 3670 loss: 1.27819
Step: 3680 loss: 1.43964
Step: 3690 loss: 1.18516
Step: 3700

Step: 6510 loss: 0.820532
Step: 6520 loss: 1.22425
Step: 6530 loss: 0.519022
Step: 6540 loss: 0.296085
Step: 6550 loss: 0.566166
Step: 6560 loss: 0.901533
Step: 6570 loss: 0.422791
Step: 6580 loss: 1.06561
Step: 6590 loss: 0.935549
Step: 6600 loss: 0.604364
Step: 6610 loss: 0.515405
Step: 6620 loss: 0.39008
Step: 6630 loss: 0.983397
Step: 6640 loss: 0.522433
Step: 6650 loss: 0.605176
Step: 6660 loss: 0.388548
Step: 6670 loss: 0.723514
Step: 6680 loss: 0.580454
Step: 6690 loss: 0.346255
Step: 6700 loss: 0.562923
Step: 6710 loss: 0.378275
Step: 6720 loss: 0.664454
Step: 6730 loss: 0.560324
Step: 6740 loss: 1.11171
Step: 6750 loss: 1.28284
Step: 6760 loss: 0.816377
Step: 6770 loss: 0.3546
Step: 6780 loss: 0.872111
Step: 6790 loss: 0.602099
Step: 6800 loss: 1.25797
Step: 6810 loss: 0.788008
Step: 6820 loss: 0.776907
Step: 6830 loss: 0.332655
Step: 6840 loss: 0.584151
Step: 6850 loss: 0.302892
Step: 6860 loss: 0.814614
Step: 6870 loss: 0.758526
Step: 6880 loss: 1.26902
Step: 6890 loss: 0.88

Step: 9690 loss: 0.328597
Step: 9700 loss: 0.416362
Step: 9710 loss: 0.644922
Step: 9720 loss: 0.860482
Step: 9730 loss: 0.380986
Step: 9740 loss: 0.26137
Step: 9750 loss: 0.514324
Step: 9760 loss: 1.00244
Step: 9770 loss: 0.216283
Step: 9780 loss: 0.9377
Step: 9790 loss: 0.519854
Step: 9800 loss: 0.689478
Step: 9810 loss: 0.24088
Step: 9820 loss: 0.476405
Step: 9830 loss: 0.470158
Step: 9840 loss: 0.694973
Step: 9850 loss: 1.13002
Step: 9860 loss: 0.383115
Step: 9870 loss: 0.647203
Step: 9880 loss: 0.238431
Step: 9890 loss: 0.481531
Step: 9900 loss: 0.326495
Step: 9910 loss: 0.309332
Step: 9920 loss: 1.03055
Step: 9930 loss: 0.600358
Step: 9940 loss: 0.727766
Step: 9950 loss: 1.26204
Step: 9960 loss: 0.594648
Step: 9970 loss: 0.367045
Step: 9980 loss: 0.767916
Step: 9990 loss: 0.666148
Step: 10000 loss: 1.3215
Saved Model
Step: 10010 loss: 0.419958
Step: 10020 loss: 0.606528
Step: 10030 loss: 0.231338
Step: 10040 loss: 0.440588
Step: 10050 loss: 0.500521
Step: 10060 loss: 0.439139
Ste

Step: 12750 loss: 0.355162
Step: 12760 loss: 0.434311
Step: 12770 loss: 0.854917
Step: 12780 loss: 0.847007
Step: 12790 loss: 0.661022
Step: 12800 loss: 0.766422
Step: 12810 loss: 0.597429
Step: 12820 loss: 0.873615
Step: 12830 loss: 1.00413
Step: 12840 loss: 0.735962
Step: 12850 loss: 0.564719
Step: 12860 loss: 0.452358
Step: 12870 loss: 0.189261
Step: 12880 loss: 0.619311
Step: 12890 loss: 0.337902
Step: 12900 loss: 0.485164
Step: 12910 loss: 0.395688
Step: 12920 loss: 0.668513
Step: 12930 loss: 0.480374
Step: 12940 loss: 0.206199
Step: 12950 loss: 0.500718
Step: 12960 loss: 0.509961
Step: 12970 loss: 0.419273
Step: 12980 loss: 0.55534
Step: 12990 loss: 0.768301
Step: 13000 loss: 0.948246
Saved Model
Step: 13010 loss: 0.479753
Step: 13020 loss: 0.118635
Step: 13030 loss: 0.683778
Step: 13040 loss: 0.841048
Step: 13050 loss: 0.393523
Step: 13060 loss: 0.558165
Step: 13070 loss: 0.246032
Step: 13080 loss: 0.525319
Step: 13090 loss: 0.524355
Step: 13100 loss: 0.370541
Step: 13110 loss: 

Step: 15800 loss: 0.417705
Step: 15810 loss: 0.814612
Step: 15820 loss: 0.505755
Step: 15830 loss: 0.649637
Step: 15840 loss: 0.261104
Step: 15850 loss: 0.529636
Step: 15860 loss: 0.129469
Step: 15870 loss: 0.252445
Step: 15880 loss: 0.34366
Step: 15890 loss: 0.299679
Step: 15900 loss: 0.125859
Step: 15910 loss: 0.368362
Step: 15920 loss: 0.424977
Step: 15930 loss: 1.15646
Step: 15940 loss: 0.680904
Step: 15950 loss: 0.70376
Step: 15960 loss: 0.297565
Step: 15970 loss: 0.591049
Step: 15980 loss: 0.786074
Step: 15990 loss: 0.28476
Step: 16000 loss: 0.715381
Saved Model


In [21]:
# training accuracy
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

training accuracy 84.0375


In [22]:
f = open(data_dir + '/test_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open(data_dir + '/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [23]:
# test 
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("test accuracy", accuracy)

test accuracy 60.405904059
